In [2]:
# Setup
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import time
import warnings
import torch.nn as nn
import pickle
from dataset_loader import *
from utils import *
import pandas as pd
from datasets import Dataset, DatasetDict

SEED = 84
torch.manual_seed(SEED)
warnings.simplefilter("ignore")

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
dataset_name = 'offensive'
ds = get_ds(dataset_name)

In [5]:
#model_name ='google/electra-small-discriminator'
model_name =  'huawei-noah/TinyBERT_General_4L_312D'
folder_name = 'tinybert'

In [ ]:
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')
sentences = [tokenizer.tokenize(s) for i, s in enumerate(sentences) if labels[i]==desired_label]
def _replace_tokens(self, removed_tokens, sentences):
    replaced_sentences = []
    for s in sentences:
        tokenized_sentence = copy.deepcopy(s)
        for i in range(len(tokenized_sentence)):
            token = tokenized_sentence[i]
            if token in removed_tokens:
                tokenized_sentence[i] = '[MASK]'
                sentence = self.tokenizer.decode(self.tokenizer.encode(tokenized_sentence))
                results = self.unmasker(sentence, top_k=2)
                for r in results:
                    if r['token_str']!=token:
                        tokenized_sentence[i] = r['token_str']
                        break

        replaced_sentences.append(tokenized_sentence)
    return replaced_sentences 

In [ ]:
set_seed()
model = load_model(model_name)
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)
train(model, tokenized_data, path=f'{folder_name}/{dataset_name}', num_train_epochs=10)
model.save_pretrained(f'{folder_name}/{dataset_name}/model')

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/huawei-noah/TinyBERT_General_4L_312D/resolve/main/config.json from cache at /home/almr16/.cache/huggingface/transformers/740f3ffeb47194ea97e78563af6f51d8fce8d8346d36d41889c75780869cf609.71ad48e6ad371e0c3ffa4982a5cc1c97e41023678e120b71ebe50addd42ec567
Model config BertConfig {
  "_name_or_path": "huawei-noah/TinyBERT_General_4L_312D",
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "structure": [],
  "transformers_version": "4.15.0",
  "type

In [9]:
def predict_sentences(sentences):
    encoded = [[101] +[tokenizer._convert_token_to_id_with_added_voc(token) for token in tokens] + [102]         
               for tokens in sentences]
    #encoded = tokenizer.encode(sentences, add_special_tokens=True, return_tensors="pt").to(device)
    to_pred = torch.tensor(encoded, device=device)
    outputs = traced_model(to_pred)[0]
    print(outputs)
    return torch.argmax(outputs, dim=1).cpu().numpy()

### save best model's folder as 'model'

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(f'{folder_name}/{dataset_name}/updated_model', num_labels=2, torchscript=True).to(device)
model = model.eval()

loading configuration file tinybert/offensive/model/config.json
Model config BertConfig {
  "_name_or_path": "tinybert/offensive/model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "problem_type": "single_label_classification",
  "structure": [],
  "torch_dtype": "float32",
  "torchscript": true,
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file tinybert/offensive/model/pytorch_model.bin
All model checkpoint weights were used when initializ

In [23]:
encoded = tokenizer.encode("i love movies", add_special_tokens=True, return_tensors="pt").to(device)
traced_model = torch.jit.trace(model, encoded)

In [24]:
torch.jit.save(traced_model, f"{folder_name}/{dataset_name}/traced.pt")

In [7]:
traced_model = torch.jit.load(f"{folder_name}/{dataset_name}/traced.pt")

In [25]:
predict_sentences([tokenizer.tokenize("i hate movies")])

tensor([[ 2.1096, -2.0074]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([0])

In [29]:
predict_sentences([tokenizer.tokenize("i love movies")])

tensor([[-2.4865,  2.3588]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([1])

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad
count_parameters(model)

In [ ]:
# when apply torchscript to models sometimes
torch._C._jit_set_texpr_fuser_enabled(False)